## Rollback

There are several strategies for handling concurrent runs in your graph. This notebook covers how to use the `rollback` option - please see the other how-to guides in the "Double Texting" directory to learn about the other methods.

First, let's import our required packages and instantiate our client, assistant, and thread.

In [ ]:
import asyncio

import httpx
from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client

In [ ]:
client = get_client()
assistant = await client.assistants.create("agent")
thread = await client.threads.create()

In [ ]:
# the first run will be interrupted
rolled_back_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]},
)
await asyncio.sleep(2)
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="rollback",
)

In [ ]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], run["run_id"])

We can see that the thread has data only from the second run

In [ ]:
state = await client.threads.get_state(thread["thread_id"])

In [ ]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in nyc?
================================== Ai Message ==================================

[{'id': 'toolu_01JzPqefao1gxwajHQ3Yh3JD', 'input': {'query': 'weather in nyc'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01JzPqefao1gxwajHQ3Yh3JD)
 Call ID: toolu_01JzPqefao1gxwajHQ3Yh3JD
  Args:
    query: weather in nyc
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.71, 'lon': -74.01, 'tz_id': 'America/New_York', 'localtime_epoch': 1718734479, 'localtime': '2024-06-18 14:14'}, 'current': {'last_updated_epoch': 1718733600, 'last_updated': '2024-06-18 14:00', 'temp_c': 29.4, 'temp_f': 84.9, 'is_day': 1, 'condition

Verify that the original, rolled back run was deleted

In [ ]:
try:
    await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
except httpx.HTTPStatusError as _:
    print("Original run was correctly deleted")

Original run was correctly deleted
